# Graph Classification

In [1]:
import pandas as pd
import numpy as np
import json
import torch
import os
from torch_geometric.data import Data, Batch, Dataset
from torch_geometric.nn import GCNConv, Sequential, Linear, global_mean_pool
import pickle
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
import torch_geometric.utils as ut
from sklearn.model_selection import train_test_split

from gcnHelpers import *


# Training on Multiple classes
#### please put the folders of the CVE's in the script's directory<br>then inside each folder put 2 subfolders, one for the nodes and the other for edges
##### example: folder 121 contains 4 subfolders (nodes_121, edges_121, feature_matrices, nodes_targets)<br>and feature_matrices and nodes_targets contain the npy files

In [2]:
# 121   122     ok
# 23    36      not ok
# 126   127     not ok
# 194   195     not ok

In [3]:
#CVES_to_train_on = {'762':4, '191':5, '134':6, '590':7}#, '23', '401', '457', '590', '690', '762', '78', '23', '36', '126', '127', '194', '195', '191']
#CVES_to_train_on = ['122', '121', '78', '190']
#CVES_to_train_on = ['762', '191', '134', '590']
#CVES_to_train_on = ['23', '126', '194', '401','690']
CVES_to_train_on = ['36', '127', '124', '195', '457']

#List containing all input data of type Data (Pytorch geometric datastructure that holds a single graph)
allData = []
numGraphs_List = []

torch.cuda.empty_cache()

for encClass, cve in enumerate(CVES_to_train_on):
    print(f"-----------LOADING CVE {cve}------------------")

    
    pathToEdges = f"D:/ClassWork/Guardista/4-Features_Extraction/{cve}/edges_{cve}"

    with open (f'{cve}/features_matrices/features_matrices_'+cve+'.npy', 'rb') as f:
        features_matrices_list = np.load(f,  allow_pickle=True)

    with open(f'{cve}/nodes_targets/nodes_targets_'+cve+'.npy', 'rb') as f:
        nodes_targets_list = np.load(f,  allow_pickle=True)

    
    print(f"num Graphs : {features_matrices_list.shape[0]}")
    numGraphs_List.append(features_matrices_list.shape[0])


    # Reading the adjacency list of each graph
    adj_Lists = []
    for i, filename in enumerate(os.listdir(pathToEdges)):
       
        try:
            f = os.path.join(pathToEdges, filename)
            df = pd.read_csv(f, header=None)
            df.dropna(inplace=True)             # <---- here is the dropping
            adj_Lists.append(df)
        except:
            print('something wrong')
        


    numVulnNodes = 0

    encoded = encClass
    # Reading the label of each node
    Targets_List = []
    for i, target in enumerate(nodes_targets_list):
        if(i == len(adj_Lists)):
            break
        numVulnNodes += sum(target)
        Targets_List.append(encoded)
            

    numNodes = 0
    # Reading the features of each node
    node_Features_List = []
    
    for i, node_feature in enumerate(features_matrices_list):
        n = np.array(node_feature,dtype='int64')
        numNodes+= n.shape[0]
        node_Features_List.append(torch.tensor(n, dtype=torch.float))


    try:
        assert (len(adj_Lists) == len(Targets_List))
        assert (len(adj_Lists) == len(node_Features_List))
    except:
        print(len(adj_Lists))
        print(len(Targets_List))
        print(len(node_Features_List))


    # Convert the edge list to use 0-based indices
    adj_Tensors = []
    for adj in adj_Lists:
        # the documentation strictly says to convert the adjacency list to a contiguous list
        adj_Tensors.append(torch.as_tensor(adj.to_numpy(), dtype=torch.long).t().contiguous())   


    #Iterate over each graph, make a Data object, then append to all our dataset
    cntCorruptData=0
    for i, adj in enumerate(adj_Tensors):
        d = Data(x=node_Features_List[i], edge_index=adj, y=torch.as_tensor(Targets_List[i]))  # <--- from the documentation.
        try:
            d.validate(raise_on_error=True)     # <--------- this line makes sure each input graph strictly follows the correct rules, to evade errors
        except:
            print('DIMENSION ERROR')
            print(f"We have features for {len(node_Features_List[i])} Nodes ")
            print(f"But the adjacency list contains {max(set(np.array(adj[0,:])))} Unique Nodes")
            cntCorruptData +=1
            continue
        allData.append(d)


    print(f"num nodes : {numNodes} of which {numVulnNodes} are vulnerable")
    print(f"number of corrupted files due to missing node features for certain nodes = {cntCorruptData}")
    print(f"total number of files {len(adj_Tensors)}")
    print(f"number of files to be trained on = {len(adj_Tensors) - cntCorruptData}\n\n")
   

-----------LOADING CVE 23------------------
num Graphs : 3313
num nodes : 8855827 of which 37034 are vulnerable
number of corrupted files due to missing node features for certain nodes = 0
total number of files 3313
number of files to be trained on = 3313


-----------LOADING CVE 126------------------
num Graphs : 1949
num nodes : 4179998 of which 9807 are vulnerable
number of corrupted files due to missing node features for certain nodes = 0
total number of files 1949
number of files to be trained on = 1949


-----------LOADING CVE 194------------------
num Graphs : 1592
num nodes : 3381463 of which 16192 are vulnerable
number of corrupted files due to missing node features for certain nodes = 0
total number of files 1592
number of files to be trained on = 1592


-----------LOADING CVE 401------------------
num Graphs : 2261
num nodes : 4553383 of which 8766 are vulnerable
number of corrupted files due to missing node features for certain nodes = 0
total number of files 2261
number of

#### Computing class weights to be used in a weighted loss function

In [4]:
total_Graphs = sum(numGraphs_List)
Class_Weights = [total_Graphs/i for i in numGraphs_List]
print(f"Weights for each class are:\n {Class_Weights}")

Weights for each class are:
 [3.1515242982191367, 5.357106208311955, 6.558417085427136, 4.617868199911544, 7.874057315233785]


### Train-Val-Test-Split and data loader

In [5]:

# Train Test Split
allData_train, allData_test = train_test_split(allData, test_size=0.2, shuffle=True)

# Train Validation Split
allData_train, allData_val = train_test_split(allData_train, test_size=0.1, shuffle=True)

unique_classes = set()

for d in allData_train:
    unique_classes.add(int(d.y.numpy()))

print(f"Unique classes {unique_classes}")


#Our Data Loader
batch_size = 5
loader = DataLoader(allData_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(allData_test, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(allData_val, batch_size=batch_size, shuffle=True)

Unique classes {0, 1, 2, 3, 4}


# GCN Class
#### anything related to the architecture

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

numClasses = len(list(unique_classes))

model = GCN(hidden_channels=128, numClasses=numClasses, numFeatures=33).to(device=device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-7)
penaltyWeights = torch.tensor( Class_Weights)  # <--- THOSE WEIGHTS ARE COMPUTED MANUALLY ACCORDING TO AN EQUATION
criterion = torch.nn.CrossEntropyLoss(weight=penaltyWeights.to(device=device))

for epoch in range(1, 13):
    train(model, loader, device, optimizer, criterion)
    train_acc = test(model, loader, device)
    val_acc = test(model, val_loader, device)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Validation Acc: {val_acc:.4f}')

# Saving the model
with open('GCN_class_g4_2.pkl', 'wb') as f:
    pickle.dump(model.cpu(), f)


GCN(
  (conv1): GCNConv(33, 128)
  (conv2): GCNConv(128, 128)
  (conv3): GCNConv(128, 128)
  (conv4): GCNConv(128, 128)
  (conv5): GCNConv(128, 128)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (lin): Linear(in_features=128, out_features=5, bias=True)
)
Epoch: 001, Train Acc: 0.3655, Validation Acc: 0.3708
Epoch: 002, Train Acc: 0.3857, Validation Acc: 0.3888
Epoch: 003, Train Acc: 0.4722, Validation Acc: 0.4450
Epoch: 004, Train Acc: 0.6034, Validation Acc: 0.5778
Epoch: 005, Train Acc: 0.7098, Validation Acc: 0.7273
Epoch: 006, Train Acc: 0.6486, Validation Acc: 0.6124
Epoch: 007, Train Acc: 0.8489, Validation Acc: 0.8732
Epoch: 008, Train Acc: 0.7171, Validation Acc: 0.7129
Epoch: 009, Train Acc: 0.9625, Validation Acc: 0.9653
Epoch: 010, Train Acc: 0.9629, Validation Acc: 0.9629
Epoch: 011, Train Acc: 0.9730, Validation Acc: 0.9797
Epoch: 012, Train Acc: 0.9557, Validation Acc: 0.9641


# Testing

In [7]:
from sklearn.metrics import classification_report

with open('GCN_class_g4_2.pkl', 'rb') as f:
    model = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device=device)

y_true = []
y_predicted = []
unique_classes = set()
for d in allData_test:
    y_true.append(d.y.cpu().numpy())
    
    pred,_ = inference(model, d.to(device=device), device=device)
    y_predicted.append(pred)


print(classification_report(y_true= y_true, y_pred= y_predicted))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       645
           1       0.99      0.90      0.94       394
           2       0.99      0.95      0.97       299
           3       0.95      0.94      0.95       469
           4       0.90      1.00      0.95       282

    accuracy                           0.96      2089
   macro avg       0.96      0.96      0.96      2089
weighted avg       0.96      0.96      0.96      2089



# Inference

### Inference on only 1 datapoint

#### You input 2 folders, one folder containing the json of nodes, the other contains the csv for edges.<br>don't mix them in one folder, bad things happen<br>if you have a npy file, please pass its abs path as this file will make the script run much faster

In [8]:
import numpy as np

pathToUser_Nodes = r"D:\ClassWork\Guardista\4-Features_Extraction\test\457_test\nodes_457_test".replace('\\', '/')
pathToUser_Edges = r"D:\ClassWork\Guardista\4-Features_Extraction\test\457_test\edges_457_test".replace('\\', '/')
outPath = r"D:\ClassWork\Guardista\4-Features_Extraction\test\457_test".replace('\\', '/')
classification, Embdding_df = InferenceGCN('GCN.pkl',pathToUser_Nodes, pathToUser_Edges, outPath, multipleFiles='true', npyPath=None)

TypeError: InferenceGCN() got multiple values for argument 'multipleFiles'

In [ ]:
print(classification[0])
print('------------------------------------------------')
print(Embdding_df.head())
print('------------------------------------------------')

: 

#### Reading the outputted CSV

In [ ]:
import pandas as pd
newdf = pd.read_csv(f"{outPath}/embeddings.csv".replace('\\', '/'), header=None, index_col=None)
newdf.head()

: 

In [ ]:


cve = '78'
outPath = 'D:/ClassWork/Guardista/4-Features_Extraction/78'.replace('\\', '/')
gcnModelPath = "D:/ClassWork/Guardista/4-Features_Extraction/GCN_class_g1".replace('\\','/')
pathToUser_Edges = "D:/ClassWork/Guardista/4-Features_Extraction/78/edges_78".replace('\\', '/')


npyPath = outPath+'/features_matrices/features_matrices_' + cve + '.npy'


classification_1, embeddings_df_1 = InferenceGCN( gcnModelPath , pathToUser_Edges=pathToUser_Edges , outputPath=outPath, multipleFiles='true', npyPath=npyPath, cve=cve)
#embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_1.to_csv(f"{outPath}/embeddings_{cve}.csv", header=False, index=False)


number of corrupted files discarded = 0


100%|██████████| 6627/6627 [01:09<00:00, 95.32it/s] 


In [ ]:
cve = '78'
outPath = 'D:/ClassWork/Guardista/4-Features_Extraction/78'.replace('\\', '/')
gcnModelPath = "D:/ClassWork/Guardista/4-Features_Extraction/GCN_class_g1_safe".replace('\\','/')
pathToUser_Edges = "D:/ClassWork/Guardista/4-Features_Extraction/78/edges_78".replace('\\', '/')


npyPath = outPath+'/features_matrices/features_matrices_' + cve + '.npy'


classification_2, embeddings_df_2 = InferenceGCN( gcnModelPath , pathToUser_Edges=pathToUser_Edges , outputPath=outPath, multipleFiles='true', npyPath=npyPath, cve=cve)
#embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_2.to_csv(f"{outPath}/embeddings_{cve}.csv", header=False, index=False)

number of corrupted files discarded = 0


100%|██████████| 6627/6627 [01:02<00:00, 106.43it/s]


In [ ]:
LabelColumn = embeddings_df_1.iloc[:, -1]
embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_2.drop(columns=(embeddings_df_2.columns)[0], inplace=True)
embeddings_df_2.drop(columns=(embeddings_df_2.columns)[-1], inplace=True)
finalDF = pd.concat([embeddings_df_1, embeddings_df_2, LabelColumn], axis=1, ignore_index=True)
print(len(finalDF.columns))
finalDF.to_csv(f"{outPath}/concatEmbeddings_{cve}.csv", header=False, index=False)

258


In [ ]:

cve = '78_safe'
outPath = 'D:/ClassWork/Guardista/4-Features_Extraction/78_safe'.replace('\\', '/')
gcnModelPath = "D:/ClassWork/Guardista/4-Features_Extraction/GCN_class_g1.pkl".replace('\\','/')
pathToUser_Edges = "D:/ClassWork/Guardista/4-Features_Extraction/78_safe/edges_78_safe".replace('\\', '/')


npyPath = outPath+'/features_matrices/features_matrices_' + cve + '.npy'


classification_1, embeddings_df_1 = InferenceGCN( gcnModelPath , pathToUser_Edges=pathToUser_Edges , outputPath=outPath, multipleFiles='true', npyPath=npyPath, cve=cve)
#embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_1.to_csv(f"{outPath}/embeddings_{cve}.csv", header=False, index=False)

number of corrupted files discarded = 0


100%|██████████| 6630/6630 [01:07<00:00, 98.28it/s] 


In [ ]:
cve = '78_safe'
outPath = 'D:/ClassWork/Guardista/4-Features_Extraction/78_safe'.replace('\\', '/')
gcnModelPath = "D:/ClassWork/Guardista/4-Features_Extraction/GCN_class_g1_safe.pkl".replace('\\','/')
pathToUser_Edges = "D:/ClassWork/Guardista/4-Features_Extraction/78_safe/edges_78_safe".replace('\\', '/')


npyPath = outPath+'/features_matrices/features_matrices_' + cve + '.npy'


classification_2, embeddings_df_2 = InferenceGCN( gcnModelPath , pathToUser_Edges=pathToUser_Edges , outputPath=outPath, multipleFiles='true', npyPath=npyPath, cve=cve)
#embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_2.to_csv(f"{outPath}/embeddings_{cve}.csv", header=False, index=False)

number of corrupted files discarded = 0


100%|██████████| 6630/6630 [01:07<00:00, 98.89it/s] 


In [ ]:
LabelColumn = embeddings_df_1.iloc[:, -1]
embeddings_df_1.drop(columns=list(embeddings_df_1.columns)[-1], inplace=True)
embeddings_df_2.drop(columns=(embeddings_df_2.columns)[0], inplace=True)
embeddings_df_2.drop(columns=(embeddings_df_2.columns)[-1], inplace=True)
finalDF = pd.concat([embeddings_df_1, embeddings_df_2, LabelColumn], axis=1, ignore_index=True)
print(len(finalDF.columns))
finalDF.to_csv(f"{outPath}/concatEmbeddings_{cve}.csv", header=False, index=False)

258
